# Estrategias de predicción con texto

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pedro9olivares/Bourbaki/blob/main/NLP-Avanzado/Estrategias_de_predicción_con_texto.ipynb)

### Introducción

En este notebook exploraremos cómo enriquecer modelos de **machine learning supervisado** para predicción de precios, usando técnicas de **Procesamiento de Lenguaje Natural (NLP)**.

El objetivo es predecir el precio de una vivienda utilizando dos fuentes de información:
1. **Datos Tabulares:** Número de recámaras, baños, estacionamientos, m2, etc.
2. **Datos de Texto:** La descripción libre que escribió el vendedor (e.g., "Hermosa casa con vista al parque...").

Para poder alimentar el texto a nuestro predictor, utilizaremos *embeddings* (o encajes). Un *embedding* es una representación vectorial de un texto, donde textos con significados similares están cerca en el espacio vectorial generado.

In [14]:
import pandas as pd
import numpy as np
import re
from openai import OpenAI
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
import getpass

In [15]:
# Configuración de API Key
key = getpass.getpass(prompt="Pega aquí tu API Key de OpenAI: ")
client = OpenAI(api_key=key)

### Carga y Limpieza de Datos
Primero, cargamos el dataset y limpiamos las columnas numéricas y de precio para asegurarnos de que el modelo pueda interpretarlas correctamente.

In [17]:
# Cargamos el dataset (tomamos una muestra pequeña para demostración)
df = pd.read_csv('houses_dataset.csv').head(15)

In [18]:
df

,url,tipo_propiedad,ciudad,zona,titulo,price,dir_texto,titulo_desc,desc,vendedor_nombre,...,Extraccion_fecha,anio_cons,piso,mantenimiento,clave_interna,venta_precio,venta,pisos,renta_precio,renta
0,https://www.easybroker.com/agent/properties/ca...,Casa,Monterrey,Monterrey,"Casa en Colinas de San Jerónimo, Monterrey","$8,500,000 EN VENTA","Colinas de San Jerónimo, Monterrey, Nuevo León",Casa en Venta Colinas de San Jeronimo,Casa amplia con excelente distribución y ubica...,Wealth Asesores,...,26:05.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.easybroker.com/agent/properties/ca...,Casa,Monterrey,Monterrey,"Casa en Bosques de Vistancia, Monterrey","$8,550,000 EN VENTA","Bosques de Vistancia, Monterrey, Nuevo León",Casa en PreVenta Carretera Nacional,PLANTA BAJA\nCochera triple con acceso indepen...,Wealth Asesores,...,26:09.5,Año de construcción: A estrenar,Cantidad de pisos en el edificio: 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.easybroker.com/agent/properties/ca...,Casa,Monterrey,Monterrey,"Casa en Mítica Residencial, Monterrey","$7,450,000 EN VENTA","Mítica Residencial, Monterrey, Nuevo León",Casa en Venta Carretera Nacional,"Estrena casa terminada con bellos acabados, se...",Wealth Asesores,...,26:12.9,Año de construcción: A estrenar,Cantidad de pisos en el edificio: 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.easybroker.com/agent/properties/ca...,Casa,Monterrey,Monterrey,"Casa en Mítica Residencial, Monterrey","$7,450,000 EN VENTA","Mítica Residencial, Monterrey, Nuevo León",Casa en Venta Carretera Nacional,"Estrena casa terminada con bellos acabados, se...",Wealth Asesores,...,26:16.0,Año de construcción: A estrenar,Cantidad de pisos en el edificio: 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.easybroker.com/agent/properties/ca...,Casa,Monterrey,Monterrey,"Casa en Mítica Residencial, Monterrey","$6,300,000 EN VENTA","Mítica Residencial, Monterrey, Nuevo León",Casa en Venta Carretera Nacional,"En colonia privada con casa club equipada, cas...",Wealth Asesores,...,26:23.5,Año de construcción: A estrenar,Cantidad de pisos en el edificio: 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,https://www.easybroker.com/agent/properties/co...,Casa,Monterrey,Monterrey,"Casa en Colinas de San Jerónimo, Monterrey","$7,000,000 EN VENTA","Colinas de San Jerónimo, Monterrey, Nuevo León",COLINAS DE SAN JERONIMO,AMPLIA CASA CON EXCELENTE UBICACION EN COLONIA...,"Quieres Comprar, Vender o Rentar? Llámanos.......",...,26:28.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,https://www.easybroker.com/agent/properties/ca...,Casa,Monterrey,Monterrey,"Casa en Las Cumbres 6 Sector D-1, Monterrey","$7,950,000 EN VENTA","Las Cumbres 6 Sector D-1, Monterrey, Nuevo León",CASA EN CUMBRES 6to SECTOR D,Residencia contemporánea TOTALMENTE EQUIPADA e...,"Quieres Comprar, Vender o Rentar? Llámanos.......",...,26:39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,https://www.easybroker.com/agent/properties/ca...,Casa,Monterrey,Monterrey,"Casa en Privadas de Lincoln, Monterrey","$1,990,000 EN VENTA","Privadas de Lincoln, Monterrey, Nuevo León",CASA REMODELADA EN PRIVADAS DE LINCOLN,preciosa casa en Privadas de Lincoln Monterrey...,"Quieres Comprar, Vender o Rentar? Llámanos.......",...,26:43.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,https://www.easybroker.com/agent/properties/am...,Casa,Monterrey,Monterrey,"Casa en Colinas de San Jerónimo 1 Sector, Mont...","$13,400,000 EN VENTA","Colinas de San Jerónimo 1 Sector, Monterrey, N...",AMPLIA CASA EN COLINAS DE SAN JERONIMO,"AMPLIA PROPIEDAD, CON VISTA AL CERRO\nTerraza ...","Quieres Comprar, Vender o Rentar? Llámanos.......",...,26:50.3,NaN,Cantidad de pisos en el edificio: 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,https://www.easybroker.com/agent/properties/ca...,Casa,Monterrey,Monterrey,"Casa en Las Cumbres 4 Sector A, Monterrey","$7,000,000 EN VENTA","Las Cumbres 4 Sector A, Monterrey, Nuevo León",CASA EN CUMBRES 4 SECTOR,VEN Y CONOCE ESTA AMPLIA CASA EN CUMBRES 4 SEC...,"Quieres Comprar, Vender o Rentar? Llámanos.......",...,26:58.5,NaN,NaN,NaN,Na

In [19]:
# Funciones de limpieza
def clean_price(val):
    if pd.isna(val): return np.nan
    match = re.search(r'\$?([\d,]+)', str(val))
    return float(match.group(1).replace(',', '')) if match else np.nan

def clean_numeric(val):
    if pd.isna(val): return 0.0
    match = re.search(r'(\d+\.?\d*)', str(val))
    return float(match.group(1)) if match else 0.0

In [20]:
# Aplicamos limpieza
df['target_price'] = df['price'].apply(clean_price)
df['n_recamaras'] = df['recamara'].apply(clean_numeric)
df['n_banio'] = df['banio'].apply(clean_numeric)
df['n_estacionamiento'] = df['estacionamiento'].apply(clean_numeric)
df['m2_construccion'] = df['metros_construccion'].apply(clean_numeric)
df['m2_terreno'] = df['metros_terreno'].apply(clean_numeric)

In [21]:
# Seleccionamos columnas relevantes y eliminamos nulos
cols_to_keep = ['desc', 'target_price', 'n_recamaras', 'n_banio', 'n_estacionamiento', 'm2_construccion', 'm2_terreno']
df_ml = df.dropna(subset=['target_price', 'desc']).copy()
df_ml = df_ml[cols_to_keep]

print(f"Dimensiones del dataset limpio: {df_ml.shape}")
df_ml.head()

Dimensiones del dataset limpio: (15, 7)


,desc,target_price,n_recamaras,n_banio,n_estacionamiento,m2_construccion,m2_terreno
0,Casa amplia con excelente distribución y ubica...,8500000.0,3.0,3.0,2.0,325.0,336.0
1,PLANTA BAJA\nCochera triple con acceso indepen...,8550000.0,3.0,4.0,3.0,303.0,225.0
2,"Estrena casa terminada con bellos acabados, se...",7450000.0,3.0,4.0,3.0,329.0,206.0
3,"Estrena casa terminada con bellos acabados, se...",7450000.0,3.0,4.0,3.0,329.0,206.0
4,"En colonia privada con casa club equipada, cas...",6300000.0,3.0,4.0,2.0,274.0,200.0


### Embeddings via OpenAI's API

En lugar de generar texto (como lo hace un modelo de chat), un modelo de embeddings convierte una cadena en un vector numérico que puede usarse para:

- búsqueda semántica
- clustering
- recomendación
- RAG (Retrieval Augmented Generation)
- comparación de similitud

Cuando llamamos al endpoint de embeddings, estamos haciendo una llamada normal al API de OpenAI, pero especializada en producir vectores en lugar de texto.

In [22]:
# Función para obtener embeddings usando OpenAI
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

**Costos**

El costo de embeddings depende únicamente de:

- número de tokens de entrada
- modelo elegido

No hay tokens de salida como en generación de texto, lo que suele hacer que embeddings sea más barato y predecible en costo.

**Modelos disponibles**

La lista actualizada de modelos, límites y recomendaciones de uso se encuentra en la documentación oficial: https://developers.openai.com/api/docs/guides/embeddings

## Estrategia 1: Predicción Híbrida (Vectores + Tabular)

En esta estrategia, aprovechamos la estructura explícita de los datos numéricos y la combinamos con la información semántica del texto.

1. **Embeddings:** Convertimos la columna `desc` (descripción) en un vector numérico de alta dimensión.
2. **Concatenación:** Unimos este vector con las columnas numéricas originales (`n_recamaras`, `m2`, etc.).
3. **Predicción:** El modelo recibe un vector extendido que contiene tanto "hechos numéricos" como "significado del texto".

In [23]:
print("Generando embeddings para las descripciones originales...")
df_ml['embedding_orig'] = df_ml['desc'].apply(lambda x: get_embedding(x))

Generando embeddings para las descripciones originales...


In [24]:
# Visualizamos cómo se ve el vector de embedding
print(f"Tamaño de un vector de embedding: {len(df_ml['embedding_orig'].iloc[0])} dimensiones")
df_ml[['desc', 'embedding_orig']].head(2)

Tamaño de un vector de embedding: 1536 dimensiones


,desc,embedding_orig
0,Casa amplia con excelente distribución y ubica...,"[0.027999509125947952, -0.01967393048107624, 0..."
1,PLANTA BAJA\nCochera triple con acceso indepen...,"[0.01095962431281805, -0.015751410275697708, 0..."


In [25]:
# Preparamos la matriz X para la Estrategia 1

# 1. Convertimos la lista de embeddings en una matriz numpy
X_emb = np.array(df_ml['embedding_orig'].tolist())

# 2. Obtenemos la matriz de características numéricas
X_num = df_ml[['n_recamaras', 'n_banio', 'n_estacionamiento', 'm2_construccion', 'm2_terreno']].values

# 3. Concatenamos horizontalmente (hstack)
X1 = np.hstack([X_emb, X_num])

In [27]:
X1

array([[ 2.79995091e-02, -1.96739305e-02,  4.65334244e-02, ...,
         2.00000000e+00,  3.25000000e+02,  3.36000000e+02],
       [ 1.09596243e-02, -1.57514103e-02,  3.07806376e-02, ...,
         3.00000000e+00,  3.03000000e+02,  2.25000000e+02],
       [ 5.36708049e-02,  1.28783903e-03,  1.50263384e-02, ...,
         3.00000000e+00,  3.29000000e+02,  2.06000000e+02],
       ...,
       [ 1.92598198e-02,  9.50745400e-03,  9.67889186e-03, ...,
         0.00000000e+00,  4.91000000e+02,  5.00000000e+02],
       [ 1.96773950e-02, -1.00303274e-02,  1.18104713e-02, ...,
         2.00000000e+00,  2.28000000e+02,  1.12000000e+02],
       [ 1.33634657e-02,  3.49935610e-03,  7.58738909e-03, ...,
         2.00000000e+00,  2.46000000e+02,  1.12000000e+02]])

In [28]:
y = df_ml['target_price'].values

print(f"Dimensiones de X_emb (Texto): {X_emb.shape}")
print(f"Dimensiones de X_num (Numérico): {X_num.shape}")
print(f"Dimensiones finales de X1 (Estrategia 1): {X1.shape}")

Dimensiones de X_emb (Texto): (15, 1536)
Dimensiones de X_num (Numérico): (15, 5)
Dimensiones finales de X1 (Estrategia 1): (15, 1541)


## Estrategia 2: Texto Unificado (Todo a Texto)

En esta estrategia, probamos una hipótesis diferente: **¿Puede el modelo entender los datos mejor si se los presentamos como narración?**

1. **Conversión a Prosa:** Utilizamos un LLM (GPT-4o-mini) para reescribir las características numéricas (ej. `3 baños`, `200 m2`) como una descripción elegante de bienes raíces.
2. **Fusión:** Concatenamos esta nueva descripción "técnica hecha prosa" con la descripción original del vendedor.
3. **Embedding Único:** Generamos un solo embedding de este texto combinado. No usamos columnas numéricas explícitas en el modelo final, solo el vector del texto completo.

In [29]:
def generate_prose(row):
    # Convertimos los datos duros en una narrativa natural
    prompt = f"""
    Convert the following property features into a natural, elegant prose description:
    - Bedrooms: {row['n_recamaras']}
    - Bathrooms: {row['n_banio']}
    - Parking: {row['n_estacionamiento']}
    - Construction: {row['m2_construccion']} m2
    - Land: {row['m2_terreno']} m2

    Make it sound like a real estate listing that blends perfectly with a description.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [30]:
# Aplicamos la generación de texto (esto puede tardar un poco dependiendo de la cantidad de datos)
print("Transformando datos numéricos a prosa...")
df_ml['prose_features'] = df_ml.apply(generate_prose, axis=1)

Transformando datos numéricos a prosa...


In [31]:
# Creamos la fuente única de información
df_ml['combined_text'] = df_ml['desc'] + " " + df_ml['prose_features']

In [32]:
pd.set_option('display.max_colwidth', 150)
df_ml[['n_recamaras', 'm2_construccion', 'prose_features', 'combined_text']].head(2)

,n_recamaras,m2_construccion,prose_features,combined_text
0,3.0,325.0,"Welcome to your dream home, a stunning residence that artfully combines modern elegance with spacious comfort. This exquisite property boasts thre...","Casa amplia con excelente distribución y ubicación.\n336 terreno\n325 construcción\n2 plantas\n3 recámaras\n3 baños Welcome to your dream home, a ..."
1,3.0,303.0,"Welcome to this exquisite property, where modern elegance meets exceptional functionality. This beautifully designed home boasts three spacious be...","PLANTA BAJA\nCochera triple con acceso independiente al patio y a la casa, concepto abierto cocina/comedor/sala, baño de visitas, lavandería y cua..."


In [33]:
print("Generando embeddings del texto unificado (Estrategia 2)...")
df_ml['embedding_prose'] = df_ml['combined_text'].apply(lambda x: get_embedding(x))

# Preparamos la matriz X2
X2 = np.array(df_ml['embedding_prose'].tolist())

print(f"Dimensiones finales de X2 (Estrategia 2): {X2.shape}")
# Nótese que aquí NO concatenamos X_num, toda la info ya está en el vector

Generando embeddings del texto unificado (Estrategia 2)...
Dimensiones finales de X2 (Estrategia 2): (15, 1536)


In [34]:
X2

array([[ 0.05019323, -0.0139449 ,  0.05136931, ...,  0.00514796,
        -0.00433415,  0.00036063],
       [ 0.03761452, -0.02032852,  0.04369956, ...,  0.01516751,
        -0.00788237,  0.0217709 ],
       [ 0.06051396, -0.00494064,  0.03867858, ...,  0.01485718,
        -0.00212168,  0.01084715],
       ...,
       [ 0.02671962,  0.00168501,  0.01650844, ...,  0.01800622,
         0.02056449,  0.00030407],
       [ 0.03612879, -0.01483434,  0.01728135, ...,  0.03180029,
         0.00350467,  0.01062004],
       [ 0.02428447, -0.00503385,  0.00831444, ...,  0.02308658,
        -0.00768282,  0.01602993]])

## Evaluación y Resultados

Utilizaremos una regresión **Ridge**. Este modelo es ideal cuando trabajamos con embeddings porque maneja muy bien la multicolinealidad y la alta dimensionalidad (muchas características).

Métrica: **MAPE (Mean Absolute Percentage Error)**. Nos dice, en promedio, qué porcentaje se desvía nuestra predicción del precio real (ej. un error del 10%).

In [35]:
def train_and_eval_mape(X, y, title):
    # Separamos en train y test (80/20)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenamos Ridge
    model = Ridge(alpha=1.0)
    model.fit(X_train, y_train)

    # Predecimos
    preds = model.predict(X_test)
    
    # Evaluamos
    mape = mean_absolute_percentage_error(y_test, preds) * 100

    print(f"--- {title} ---")
    print(f"Dimensiones de entrada: {X.shape}")
    print(f"Error (MAPE): {mape:.2f}%")
    print("-" * 30)
    return mape

In [36]:
# Ejecutamos la comparación
mape_mod1 = train_and_eval_mape(X1, y, "Estrategia 1: Embeddings + Columnas Numéricas")
mape_mod2 = train_and_eval_mape(X2, y, "Estrategia 2: Texto Unificado (Solo Embeddings)")

print(f"Diferencia de rendimiento: {mape_mod2 - mape_mod1:.2f} puntos porcentuales")

--- Estrategia 1: Embeddings + Columnas Numéricas ---
Dimensiones de entrada: (15, 1541)
Error (MAPE): 23.36%
------------------------------
--- Estrategia 2: Texto Unificado (Solo Embeddings) ---
Dimensiones de entrada: (15, 1536)
Error (MAPE): 8.99%
------------------------------
Diferencia de rendimiento: -14.37 puntos porcentuales
